# LangChain Tutorial: Essential Concepts for Beginners

This notebook demonstrates 5 key LangChain concepts with practical examples designed for university students.

## What You'll Learn:
1. **Models** - AI systems that understand and generate text
2. **Prompts** - Instructions that guide AI behavior
3. **Chains** - Connect components to create workflows
4. **Embeddings & Vectors** - Convert text to numerical representations
5. **Agents** - AI that can use tools and make decisions

---

## Setup: Install Required Packages

Run this cell first to install all necessary packages:

In [ ]:
# Install required packages
!pip install langchain
!pip install openai
!pip install chromadb
!pip install langchain-openai
!pip install langchain-community
!pip install tiktoken
!pip install duckduckgo-search

## Setup: Import Libraries and Set API Key

**Important**: Get your OpenAI API key from [platform.openai.com/api-keys](https://platform.openai.com/api-keys)

In [ ]:
import os
from langchain_openai import OpenAI, ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.vectorstores import Chroma
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import DuckDuckGoSearchRun
import warnings
warnings.filterwarnings('ignore')

# Set your OpenAI API key here
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Set up OpenAI API key based on environment
if IN_COLAB:
    # For Google Colab: use the secure input method
    from google.colab import userdata

    try:
        # Try to get API key from Colab secrets first
        openai_api_key = userdata.get('OPENAI_API_KEY')
        if openai_api_key:
            os.environ["OPENAI_API_KEY"] = openai_api_key
            print("✅ API key loaded from Google Colab secrets!")
        else:
            # If not in secrets, prompt user to enter it
            from getpass import getpass
            print("OpenAI API key not found in Colab secrets.")
            os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
            print("✅ API key set from input")
    except Exception as e:
        print(f"Note: {e}")
        print("Enter your OpenAI API key below:")
        os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key: ")
else:
    # For local environment: try to load from .env file
    try:
        from dotenv import load_dotenv
        load_dotenv()
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print("✅ API key loaded from .env file")
        else:
            print("⚠️ No API key found in .env file. Using fallback value.")
            os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    except ImportError:
        print("⚠️ python-dotenv not installed. Using fallback value.")
        os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Check if API key is properly set
if os.environ.get("OPENAI_API_KEY") in [None, "", "your-api-key-here"]:
    print("⚠️ WARNING: Please set your OpenAI API key before running the examples!")
    print("Get your API key from: https://platform.openai.com/api-keys")
    if IN_COLAB:
        print("For Colab: Use Secrets to securely store your API key")
        print("  1. Click on the 🔑 icon in the left sidebar")
        print("  2. Add a new secret with name 'OPENAI_API_KEY'")
        print("  3. Run this cell again")
    else:
        print("For local use: Create a .env file with OPENAI_API_KEY=your-key-here")
else:
    print("✅ API key is set! Ready to proceed.\n")

print("LangChain Tutorial: 5 Essential Concepts")
print("=" * 60)

---
# CONCEPT 1: MODELS

Models are AI systems that can understand and generate text. Think of them as very smart assistants that can help with various tasks.

There are different types of models:
- **Basic models**: Good for simple text completion
- **Chat models**: Better for conversations

In [ ]:
# Initialize different types of models
basic_llm = OpenAI(temperature=0.7, max_tokens=100)
chat_model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")

print("Example 1a: Basic Text Generation")
question = "What are the benefits of learning programming?"
response = basic_llm.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}\n")

In [ ]:
# Example with chat model
print("Example 1b: Chat-style Conversation")
messages = [
    SystemMessage(content="You are a helpful tutor explaining concepts simply."),
    HumanMessage(content="Explain what machine learning is in simple terms.")
]
chat_response = chat_model.invoke(messages)
print(f"Chat Response: {chat_response.content}")

### Try It Yourself:
- Change the `temperature` parameter (0 = focused, 1 = creative)
- Try different questions or topics
- Experiment with different system messages

---
# CONCEPT 2: PROMPTS

Prompts are like instructions you give to the AI model. Good prompts lead to better, more useful responses.

**Prompt Templates** let you create reusable prompts with variables (like fill-in-the-blank forms).

In [ ]:
print("Example 2a: Basic Prompt Template")

# Create a reusable prompt template
simple_template = PromptTemplate(
    input_variables=["topic", "audience"],
    template="Explain {topic} to {audience} in a simple and engaging way."
)

# Use the template with different inputs
topic_prompt = simple_template.format(topic="photosynthesis", audience="high school students")
print(f"Generated Prompt: {topic_prompt}")

# Get AI response
explanation = basic_llm.invoke(topic_prompt)
print(f"AI Response: {explanation}")

In [ ]:
print("Example 2b: Chat Prompt Template")

# More structured chat template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly study buddy helping with {subject}."),
    ("human", "I need help understanding {concept}. Can you give me a simple example?")
])

# Format and use the template
formatted_chat_prompt = chat_template.format_messages(
    subject="mathematics",
    concept="derivatives"
)

print("Chat Prompt Messages:")
for msg in formatted_chat_prompt:
    print(f"  {msg.type}: {msg.content}")

chat_explanation = chat_model.invoke(formatted_chat_prompt)
print(f"\nAI Response: {chat_explanation.content}")

### Try It Yourself:
- Create your own prompt template for generating study notes
- Try different subjects and concepts
- Experiment with different instruction styles

---
# CONCEPT 3: CHAINS

Chains connect different components (models, prompts) to create workflows. Think of them as assembly lines for AI tasks.

You can create:
- **Simple chains**: One prompt + one model
- **Sequential chains**: Multiple steps that build on each other

In [ ]:
print("Example 3a: Basic LLM Chain")

# Create a prompt for generating study questions
question_template = PromptTemplate(
    input_variables=["subject", "difficulty"],
    template="Generate 3 {difficulty} level study questions about {subject}."
)

# Create a chain (prompt + model)
question_chain = LLMChain(llm=basic_llm, prompt=question_template)

# Use the chain
study_questions = question_chain.run(subject="Python programming", difficulty="beginner")
print(f"Generated Study Questions:\n{study_questions}")

In [ ]:
print("Example 3b: Sequential Chain (Multi-step)")

# Step 1: Generate a topic summary
summary_template = PromptTemplate(
    input_variables=["topic"],
    template="Write a brief summary of {topic} in 2-3 sentences."
)
summary_chain = LLMChain(llm=basic_llm, prompt=summary_template)

# Step 2: Create quiz questions from the summary
quiz_template = PromptTemplate(
    input_variables=["summary"],
    template="Based on this summary: {summary}\n\nCreate 2 multiple choice questions."
)
quiz_chain = LLMChain(llm=basic_llm, prompt=quiz_template)

# Combine chains into a sequence
overall_chain = SimpleSequentialChain(
    chains=[summary_chain, quiz_chain],
    verbose=True  # Shows intermediate steps
)

# Run the sequential chain
result = overall_chain.run("renewable energy")
print(f"\nFinal Result:\n{result}")

### Try It Yourself:
- Create a chain that summarizes text and then translates it
- Build a study aid that generates notes, then creates flashcards
- Try different topics with the sequential chain

---
# CONCEPT 4: EMBEDDINGS AND VECTORS

Embeddings convert text into numbers (vectors) that represent meaning. Similar texts have similar numbers.

This enables:
- **Semantic search**: Find similar content by meaning, not just keywords
- **Knowledge bases**: Store and retrieve information intelligently

In [ ]:
print("Example 4a: Creating Text Embeddings")

# Initialize embeddings
embeddings = OpenAIEmbeddings()

# Sample texts
texts = [
    "Python is a programming language",
    "Java is also a programming language",
    "Cats are cute animals",
    "Dogs are loyal pets"
]

# Create embeddings
print("Creating embeddings for sample texts...")
text_embeddings = embeddings.embed_documents(texts)

print(f"Number of texts: {len(texts)}")
print(f"Embedding dimension: {len(text_embeddings[0])}")
print(f"First few numbers of first embedding: {text_embeddings[0][:5]}")
print("\nNotice: Similar texts (programming languages) will have similar embeddings!")

In [ ]:
print("Example 4b: Vector Store for Similarity Search")

# Create a simple knowledge base
knowledge_base = [
    "LangChain is a framework for building AI applications",
    "Python is popular for data science and machine learning",
    "Machine learning models can recognize patterns in data",
    "Natural language processing helps computers understand text",
    "Vectors are mathematical representations of data"
]

# Create a vector store (this creates a local database)
vectorstore = Chroma.from_texts(
    knowledge_base,
    embeddings,
    persist_directory="./chroma_db"  # Creates a local folder
)

print("Note: This creates a 'chroma_db' folder in your current directory.")
print("You can delete it later if you want to clean up.\n")

# Search for similar content
query = "What is machine learning?"
similar_docs = vectorstore.similarity_search(query, k=2)

print(f"Query: {query}")
print("Most similar documents:")
for i, doc in enumerate(similar_docs, 1):
    print(f"{i}. {doc.page_content}")

### Try It Yourself:
- Add your own documents to the knowledge base
- Try different search queries
- Create a vector store with course notes or textbook content

---
# CONCEPT 5: AGENTS

Agents are AI systems that can decide which tools to use to solve problems. They can search the web, do calculations, access databases, and more!

Think of agents as AI assistants that can actually **do things** in the real world.

In [ ]:
print("Example 5: Web Search Agent")

# Create a search tool
search = DuckDuckGoSearchRun()

# Define tools for the agent
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for searching current information on the internet"
    )
]

# Create an agent
agent = initialize_agent(
    tools,
    chat_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # Shows the agent's thinking process
)

print("Agent Task: Find current information about a topic")
try:
    response = agent.run("What are the latest developments in renewable energy technology?")
    print(f"\nAgent Response: {response}")
except Exception as e:
    print(f"Note: Web search might be limited in some environments. Error: {e}")
    print("In a full setup, the agent would search the web and provide current information.")

### Try It Yourself:
- Ask the agent different questions that require web search
- Create agents with different tools (calculator, database access, etc.)
- Design an agent that can help with homework questions

---
# TUTORIAL COMPLETE!

## Summary of Concepts:

### 1. **MODELS**: AI systems that understand and generate text
- Use `OpenAI`, `ChatOpenAI` for different types of tasks
- Temperature controls creativity (0 = focused, 1 = creative)

### 2. **PROMPTS**: Instructions that guide AI behavior  
- `PromptTemplate` for reusable prompts with variables
- `ChatPromptTemplate` for conversation-style interactions

### 3. **CHAINS**: Connect components to create workflows
- `LLMChain` combines prompts and models
- Sequential chains for multi-step processes

### 4. **EMBEDDINGS**: Convert text to numerical representations
- Similar texts have similar embeddings
- Vector stores enable semantic search

### 5. **AGENTS**: AI that can use tools and make decisions
- Can search web, do calculations, access databases
- Combine reasoning with external capabilities

---

## Next Steps:
- Get an OpenAI API key from [platform.openai.com/api-keys](https://platform.openai.com/api-keys)
- Replace "your-api-key-here" with your actual API key
- Try modifying the prompts and see how outputs change
- Experiment with different model parameters
- Build your own simple AI application using these concepts!

## Cleanup (Optional):
- Delete the 'chroma_db' folder created during the vector store example
- You can recreate it anytime by running that section again

---

## Practice Exercises:
1. Create a prompt template for generating study notes
2. Build a chain that summarizes text and then translates it 
3. Create a vector store with your own documents
4. Design an agent that can help with homework questions

**Happy learning!**